In [1]:
import pandas as pd
import numpy as np
from utils import *


In [2]:
#pip install sentence-transformers

Load the datasets - Mentor/Mentee/Circle

In [2]:
Mentee_df = pd.read_excel('../data/eval/Mentee-obfuscated-dataset.xlsx', engine='openpyxl')

In [3]:
Mentee_df.columns

Index(['ID', 'Start time', 'Completion time', 'Email', 'Name', 'First Name',
       'Last Name', 'Email Address ', 'Job Title', 'Grade Level',
       'Organziation', 'Department?', 'Business Unit?', 'Time Zone',
       'Do you meet the following eligibility requirements: Salary Grade 25+ with at least one year UHG tenure (by program commencement). Must have at least meets expectations on last MAP review and canno...',
       'Have you participated in the WiADS mentorship program in the past? ',
       'Select your preferred mentorship circle type.',
       'Are you a WiADS Member?',
       'What do you hope to gain from the mentor circle program?',
       'Sort these 10 common language of leadership practices from high, middle, low based on your observed areas of strengths and opportunities. ',
       'Choose your top 3 areas of interest.'],
      dtype='object')

In [5]:
Mentor_df = pd.read_excel('../data/eval/Mentor-obfuscated-dataset.xlsx', engine='openpyxl')

In [6]:
Mentor_df.columns

Index(['Mentor_ID', 'Mentor_FirstName', 'Mentor_LastName', 'Mentor_Email',
       'Mentor_JobTitle ', 'Mentor_GradeLevel', 'Mentor_Organization',
       'Mentor_Department', 'Mentor_BusinessUnit', 'Mentor_TimeZone',
       'Mentor_circle', 'Mentor_Interest', 'Mentor_CLL', 'Mentor_Goal',
       'Mentor_Interestcount', 'Mentor_Eligibility', 'Mentor_wiads_past ',
       'Mentor_mentor_past'],
      dtype='object')

In [7]:
Circle_df = pd.read_csv('../data/eval/ganguri/MatchedMentees.csv.xlsx')

In [8]:
#Circle_df.head()

In [9]:
list(Circle_df.columns)

['Circle',
 'Mentee_ID',
 'Mentee_First_Name',
 'Mentee_Last_Name',
 'Mentee_Email',
 'Mentee_Title',
 'Mentee_GradeLevel',
 'Mentee_Organization',
 'Mentee_Department',
 'Mentee_BusinessUnit',
 'Mentee_TimeZone',
 'Mentee_Circle',
 'Mentee_Goal',
 'Mentee_Interest',
 'Mentee_CLL',
 'Mentor_ID',
 'Mentor_FirstName',
 'Mentor_LastName',
 'Mentor_Email',
 'Mentor_JobTitle\xa0',
 'Mentor_GradeLevel',
 'Mentor_Organization',
 'Mentor_Department',
 'Mentor_BusinessUnit',
 'Mentor_TimeZone',
 'Mentor_Circle',
 'Mentor_interest',
 'Mentor_CLL',
 'Mentor_Goal']

Select only 3 columns

In [10]:
Circle_basic_df = Circle_df[['Circle','Mentor_ID','Mentee_ID']]

In [11]:
#Circle_basic_df.head()
n = Circle_df.Mentee_Email.nunique()
  
print("No.of.unique values ",n)

No.of.unique values  15


In [12]:
Cicle_basic_sorted_df = Circle_basic_df.sort_values(by=['Circle','Mentor_ID','Mentee_ID'], inplace=False, ignore_index= True)

For each circle row, get the mentor and mentee information and create a detail dataframe

• Organization/Department - Mentor and Mentee should not be in same Department. 
• Time Zones – Fit circles into similar time zones. Find difference of timezone between mentor and mentee
• Grade Levels/Mentor Circle Levels – Anyone Grade level less than 28 should not be considered a mentor.
• CLL Sort – Mentors strengths match the Mentees growth areas.
• Areas of interest – Maximum match on areas of interest within a circle.
• Circle size –The number of mentees a mentor has. The circles should be close in size.
• Mentor grade error if mentor grade < 28




In [13]:
Result_Dictionary={}
Result_DF = pd.DataFrame()

In [14]:
Cicle_basic_sorted_dfdict = Cicle_basic_sorted_df.to_dict('records')
for  circle_row in Cicle_basic_sorted_dfdict:
    mentor_row = Mentor_df[Mentor_df['Mentor_ID'] ==circle_row['Mentor_ID'] ]
    mentee_row = Mentee_df[Mentee_df['Mentee_ID'] ==circle_row['Mentee_ID'] ]
    
    result = validate_mentormentee(mentor_row, mentee_row, circle_row)
    result['Mentor_Email'] = mentor_row['Mentor_Email'].iloc[0]
    result['Mentee_Email'] = mentee_row['Mentee_Email'].iloc[0]
    Result_DF = Result_DF.append(result, ignore_index=True)
    

In [15]:
Result_DF.head()

,Circle,Goal_Similarity,Mentee_Department,Mentee_Email,Mentee_Grade,Mentee_ID,Mentee_TimeZone,Mentee_TimeZoneNum,MentorMentee_CLL_Growth,MentorMentee_Common_Interests,MentorMentee_DeptError,MentorMentee_GradeError,Mentor_Department,Mentor_Eligible,Mentor_Email,Mentor_Grade,Mentor_ID,Mentor_Mentee_TimeZoneDist,Mentor_TimeZone,Mentor_TimeZoneNum
0,1.0,0.4033,Ancillary,First_4.Last_4@uvw.com,27.0,4.0,EST,3.0,0.0,0.0,0.0,1.0,Specialty benefits,0.0,First_1.Last_1@corpitsvcs.com,27.0,1.0,0.0,EST,3.0
1,1.0,0.4244,Specialty benefits,First_5.Last_5@uvw.com,28.0,5.0,PST,6.0,1.0,1.0,1.0,1.0,Specialty benefits,0.0,First_1.Last_1@corpitsvcs.com,27.0,1.0,3.0,EST,3.0
2,1.0,0.0213,Ancillary,First_5.Last_5@uvw.com,27.0,6.0,EST,3.0,0.0,0.0,0.0,1.0,Specialty benefits,0.0,First_1.Last_1@corpitsvcs.com,27.0,1.0,0.0,EST,3.0
3,1.0,0.4541,Ancillary,First_6.Last_6@uvw.com,27.0,7.0,IST,0.0,2.0,2.0,0.0,1.0,Specialty benefits,0.0,First_1.Last_1@corpitsvcs.com,27.0,1.0,3.0,EST,3.0
4,1.0,0.2395,Government Programs,First_7.Last_7@uvw.com,27.0,8.0,CST,4.0,1.0,1.0,0.0,1.0,Specialty benefits,0.0,First_1.Last_1@corpitsvcs.com,27.0,1.0,1.0,EST,3.0


Create a summary dataframe  
• Count of members in cicle
• Min Max on interests, CLL sort
• Count of departments in cicle

In [16]:

# Groupby multiple columns & multiple aggregations
grouped = Result_DF.groupby('Circle').aggregate({'Mentor_ID':['count','nunique'],'Mentee_ID':['count','nunique'],'Mentee_TimeZoneNum':['min','max'], 'Mentor_Eligible':'sum', 'MentorMentee_GradeError':'sum','MentorMentee_DeptError':'sum','Mentee_Department':'nunique','Goal_Similarity':['min','max'],'Mentee_Grade':['min','max'],'Mentee_TimeZoneNum':['min','max'],'Mentor_Mentee_TimeZoneDist':['min','max'],'MentorMentee_CLL_Growth':['min','max']})
grouped.head()


Mentor_ID         Mentee_ID         Mentee_TimeZoneNum       \
           count nunique     count nunique                min  max   
Circle                                                               
1.0           10       2        10      10                0.0  6.0   
2.0            5       1         5       5                3.0  6.0   
3.0            1       1         1       1                3.0  3.0   

       Mentor_Eligible MentorMentee_GradeError MentorMentee_DeptError  \
                   sum                     sum                    sum   
Circle                                                                  
1.0                0.0                     8.0                    1.0   
2.0                0.0                     3.0                    0.0   
3.0                1.0                     0.0                    0.0   

       Mentee_Department Goal_Similarity         Mentee_Grade        \
                 nunique             min     max          min   max   
Circle                                                                
1.0                    8          0.0213  0.4821         25.0  29.0   
2.0                    5          0.1314  0.3878         26.0  29.0   
3.0                    1          0.5207  0.5207         25.0  25.0   

       Mentor_Mentee_TimeZoneDist      MentorMentee_CLL_Growth       
                              min  max                     min  max  
Circle                                                               
1.0                           0.0  3.0                     0.0  2.0  
2.0                           0.0  2.0                     0.0  1.0  
3.0                           0.0  0.0                     2.0  2.0

In [17]:
regroup_DF = grouped.reset_index()
regroup_DF.head()

Circle Mentor_ID         Mentee_ID         Mentee_TimeZoneNum       \
             count nunique     count nunique                min  max   
0    1.0        10       2        10      10                0.0  6.0   
1    2.0         5       1         5       5                3.0  6.0   
2    3.0         1       1         1       1                3.0  3.0   

  Mentor_Eligible MentorMentee_GradeError MentorMentee_DeptError  \
              sum                     sum                    sum   
0             0.0                     8.0                    1.0   
1             0.0                     3.0                    0.0   
2             1.0                     0.0                    0.0   

  Mentee_Department Goal_Similarity         Mentee_Grade        \
            nunique             min     max          min   max   
0                 8          0.0213  0.4821         25.0  29.0   
1                 5          0.1314  0.3878         26.0  29.0   
2                 1          0.5207  0.5207         25.0  25.0   

  Mentor_Mentee_TimeZoneDist      MentorMentee_CLL_Growth       
                         min  max                     min  max  
0                        0.0  3.0                     0.0  2.0  
1                        0.0  2.0                     0.0  1.0  
2                        0.0  0.0                     2.0  2.0

Create high level statistics for easier evaluation of bonus and penalty points

Penalty points:
    Circle size > 10
    Mentor and mentee are in same department
    Number of mentees not matched to any mentor
Bonus Points
    Timezone 
    Grade Level distance <= 3
    Drop mentors with grade level < 28
      
      
• Max - Min circle size
• Circle Size > 10
• Number of mentees not in any circle
• Number of mentors not in any circle
• Duplicate entry for mentee

In [18]:
Summary_Results = createSummary(Result_DF, regroup_DF)
Summary_Results['Unassigned Mentees']=";".join(set(Mentee_df.Mentee_Email).difference(set(Circle_df.Mentee_Email)))
Summary_Results['Unassigned Mentors']=";".join(set(Mentor_df.Mentor_Email).difference(set(Circle_df.Mentor_Email)))


In [19]:
Summary_Results_DF = pd.DataFrame.from_dict(data=Summary_Results, orient='index', columns=['Result'])
Summary_Results_DF.tail(5)


,Result
Big TimeZone Diffence between mentor and mentees,Yes
Mentor mentee of same department assigned to same circle,Yes
Not enough variation in departments among mentees,Yes
Unassigned Mentees,First_38.Last_38@uvw.com;First_61.Last_61@uvw....
Unassigned Mentors,First_13.Last_13@obxic.com;First_11.Last_11@ob...


In [20]:
Summary_Results_DF.columns

Index(['Result'], dtype='object')

In [21]:
Summary_Results_DF.index = Summary_Results_DF.index.set_names('Category')
 
Summary_Results_DF.reset_index().head()

,Category,Result
0,Mentee Assigned More Than Once,2
1,Uneven Circle Size,Yes
2,Different Mentor assigned to a circle,Yes
3,Big TimeZone Diffence among mentees,Yes
4,Big TimeZone Diffence between mentor and mentees,Yes


In [22]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_multiple.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
Summary_Results_DF.to_excel(writer, sheet_name='highlevel', header='False')
regroup_DF.to_excel(writer, sheet_name='summary')
Result_DF.to_excel(writer, sheet_name='detail')

writer.close()